In [2]:
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
import csv
import sys
import numpy as np

In [3]:
!pip install gql[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [5]:
import math

In [6]:
dai_usdc100_id = "0x5777d92f208679db4b9778590fa3cab3ac9e2168"
usdc_WETH_3000_id = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"
wbtc_WETH_3000_id = "0xcbcdf9626bc03e24f779434178a73a0b4bad62ed"
UNI_WETH_3000_id = "0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801"


In [7]:
def getLiguidityDistribuation(pool_id,pool_name):
  TICK_BASE = 1.0001

  pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""

  tick_query = """query get_ticks($num_skip: Int, $pool_id: ID!) {
  ticks(skip: $num_skip, where: {pool: $pool_id}) {
    tickIdx
    liquidityNet
  }
}"""
  def tick_to_price(tick):
    return TICK_BASE ** tick
  def fee_tier_to_tick_spacing(fee_tier):
    return {
        100: 1,
        500: 10,
        3000: 60,
        10000: 200
    }.get(fee_tier, 60)

  client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
        verify=True,
        retries=5,
    ))
  # get pool info
  try:
    variables = {"pool_id": pool_id}
    response = client.execute(gql(pool_query), variable_values=variables)

    if len(response['pools']) == 0:
        print("pool not found")
        exit(-1)

    pool = response['pools'][0]
    current_tick = int(pool["tick"])
    tick_spacing = fee_tier_to_tick_spacing(int(pool["feeTier"]))

    token0 = pool["token0"]["symbol"]
    token1 = pool["token1"]["symbol"]
    decimals0 = int(pool["token0"]["decimals"])
    decimals1 = int(pool["token1"]["decimals"])
  except Exception as ex:
    print("got exception while querying pool data:", ex)
    exit(-1)
  # get tick info
  tick_mapping = {}
  num_skip = 0
  try:
    while True:
        print("Querying ticks, num_skip={}".format(num_skip))
        variables = {"num_skip": num_skip, "pool_id": pool_id}
        response = client.execute(gql(tick_query), variable_values=variables)

        if len(response["ticks"]) == 0:
            break
        num_skip += len(response["ticks"])
        for item in response["ticks"]:
            tick_mapping[int(item["tickIdx"])] = int(item["liquidityNet"])
  except Exception as ex:
    print("got exception while querying tick data:", ex)
    exit(-1)
      
  # Start from zero; if we were iterating from the current tick, would start from the pool's total liquidity
  liquidity = 0


  # Find the boundaries of the price range
  min_tick = min(tick_mapping.keys())
  max_tick = max(tick_mapping.keys())
  print (min_tick)
  print (max_tick)



  # Compute the tick range. This code would work as well in Python: `current_tick // tick_spacing * tick_spacing`
  # However, using floor() is more portable.
  current_range_bottom_tick = math.floor(current_tick / tick_spacing) * tick_spacing

  current_price = tick_to_price(current_tick)
  adjusted_current_price = current_price / (10 ** (decimals1 - decimals0))
  print (adjusted_current_price)


  # Sum up all tokens in the pool
  total_amount0 = 0
  total_amount1 = 0


  # Iterate over the tick map starting from the bottom
  invert_price = False
  storage = []
  tick = min_tick
  while tick <= max_tick:
    tmp_storage = []
    liquidity_delta = tick_mapping.get(tick, 0)
    liquidity += liquidity_delta


    tmp_storage.append(tick)

    #adding liquidity_delta
    tmp_storage.append(liquidity_delta)

    price = tick_to_price(tick)
    adjusted_price = price / (10 ** (decimals1 - decimals0))
    if invert_price:
        adjusted_price = 1 / adjusted_price
        tokens = "{} for {}".format(token0, token1)
    else:
        tokens = "{} for {}".format(token1, token0)

    should_print_tick = liquidity != 0
    #adding adjusted_price
    tmp_storage.append(adjusted_price)
    tmp_storage.append(pool_name)
    if should_print_tick:
        print("ticks=[{}, {}], bottom tick price={:.6f} {}".format(tick, tick + tick_spacing, adjusted_price, tokens))

    # Compute square roots of prices corresponding to the bottom and top ticks
    bottom_tick = tick
    top_tick = bottom_tick + tick_spacing
    sa = tick_to_price(bottom_tick // 2)
    sb = tick_to_price(top_tick // 2)

    if tick < current_range_bottom_tick:
        # Compute the amounts of tokens potentially in the range
        amount1 = liquidity * (sb - sa)
        amount0 = amount1 / (sb * sa)

        

        # Only token1 locked
        total_amount1 += amount1

        if should_print_tick:
            adjusted_amount0 = amount0 / (10 ** decimals0)
            adjusted_amount1 = amount1 / (10 ** decimals1)
            tmp_storage.append(adjusted_amount0)
            tmp_storage.append(adjusted_amount1)
            print("        {:.2f} {} locked, potentially worth {:.2f} {}".format(adjusted_amount1, token1, adjusted_amount0, token0))

    elif tick == current_range_bottom_tick:
        # Always print the current tick. It normally has both assets locked
        print("        Current tick, both assets present!")
        print("        Current price={:.6f} {}".format(1 / adjusted_current_price if invert_price else adjusted_current_price, tokens))

        # Print the real amounts of the two assets needed to be swapped to move out of the current tick range
        current_sqrt_price = tick_to_price(current_tick / 2)
        amount0actual = liquidity * (sb - current_sqrt_price) / (current_sqrt_price * sb)
        amount1actual = liquidity * (current_sqrt_price - sa)
        adjusted_amount0actual = amount0actual / (10 ** decimals0)
        adjusted_amount1actual = amount1actual / (10 ** decimals1)

        tmp_storage.append(adjusted_amount0actual)
        tmp_storage.append(adjusted_amount1actual)

        total_amount0 += amount0actual
        total_amount1 += amount1actual

        print("        {:.2f} {} and {:.2f} {} remaining in the current tick range".format(
            adjusted_amount0actual, token0, adjusted_amount1actual, token1))


    else:
        # Compute the amounts of tokens potentially in the range
        amount1 = liquidity * (sb - sa)
        amount0 = amount1 / (sb * sa)

        # Only token0 locked
        total_amount0 += amount0

        if should_print_tick:
            adjusted_amount0 = amount0 / (10 ** decimals0)
            adjusted_amount1 = amount1 / (10 ** decimals1)

            tmp_storage.append(adjusted_amount0)
            tmp_storage.append(adjusted_amount1)

            print("        {:.2f} {} locked, potentially worth {:.2f} {}".format(adjusted_amount0, token0, adjusted_amount1, token1))

    tick += tick_spacing
    total_amount0_adj = total_amount0 / 10 ** decimals0
    total_amount1_adj = total_amount1 / 10 ** decimals1
    
    tmp_storage.append(total_amount0_adj)
    tmp_storage.append(total_amount1_adj)
    storage.append(tmp_storage)
    
  df = pd.DataFrame(storage, columns = ['tick','liquidity', 'price','pool_id','amount0','amount1','total-amount0','total-amount1'])
  return df


In [8]:
usdc_WETH_3000_df = getLiguidityDistribuation(usdc_WETH_3000_id, "USDC/WETH-0.3")
usdc_WETH_3000_df

Streaming output truncated to the last 5000 lines.
ticks=[737220, 737280], bottom tick price=103623211700425949184.000000 WETH for USDC
        0.00 USDC locked, potentially worth 66142889381.92 WETH
ticks=[737280, 737340], bottom tick price=104246788652520521728.000000 WETH for USDC
        0.00 USDC locked, potentially worth 66341606040.35 WETH
ticks=[737340, 737400], bottom tick price=104874118125008977920.000000 WETH for USDC
        0.00 USDC locked, potentially worth 66540919713.99 WETH
ticks=[737400, 737460], bottom tick price=105505222699561844736.000000 WETH for USDC
        0.00 USDC locked, potentially worth 66740832196.48 WETH
ticks=[737460, 737520], bottom tick price=106140125093740232704.000000 WETH for USDC
        0.00 USDC locked, potentially worth 66941345286.83 WETH
ticks=[737520, 737580], bottom tick price=106778848161813413888.000000 WETH for USDC
        0.00 USDC locked, potentially worth 67142460789.51 WETH
ticks=[737580, 737640], bottom tick price=1074214148955

,tick,liquidity,price,pool_id,amount0,amount1,total-amount0,total-amount1
0,-887220,1150097624730994,2.954278e-51,USDC/WETH-0.3,6.338074e+25,1.878069e-25,0.000000e+00,1.878069e-25
1,-887160,98653390708394,2.972056e-51,USDC/WETH-0.3,6.861130e+25,2.045293e-25,0.000000e+00,3.923362e-25
2,-887100,0,2.989942e-51,USDC/WETH-0.3,6.840578e+25,2.051438e-25,0.000000e+00,5.974799e-25
3,-887040,0,3.007934e-51,USDC/WETH-0.3,6.820088e+25,2.057601e-25,0.000000e+00,8.032400e-25
4,-886980,0,3.026035e-51,USDC/WETH-0.3,6.799659e+25,2.063783e-25,0.000000e+00,1.009618e-24
...,...,...,...,...,...,...,...,...
29570,886980,0,3.304654e+26,USDC/WETH-0.3,3.563599e-13,1.181184e+14,6.589638e+07,9.140764e+04
29571,887040,0,3.324541e+26,USDC/WETH-0.3,3.552925e-13,1.184733e+14,6.589638e+07,9.140764e+04
29572,887100,0,3.344547e+26,USDC/WETH-0.3,3.542282e-13,1.188292e+14,6.589638e+07,9.140764e+04
29573,887160,0,3.364674e+26,USDC/WETH-0.3,3.531672e-13,1.191862e+14,6.589638e+07,9.140764e+04


In [12]:
usdc_WETH_3000_df.to_csv('Ticks_Anylsis_USDC_WETH.csv',index=False)

In [9]:
wbtc_WETH_3000_df = getLiguidityDistribuation(wbtc_WETH_3000_id,"WBTC/WETH-0.3")
wbtc_WETH_3000_df

Streaming output truncated to the last 5000 lines.
ticks=[737220, 737280], bottom tick price=10362321170042594525184.000000 WETH for WBTC
        0.00 WBTC locked, potentially worth 2179456029.02 WETH
ticks=[737280, 737340], bottom tick price=10424678865252051320832.000000 WETH for WBTC
        0.00 WBTC locked, potentially worth 2186003886.60 WETH
ticks=[737340, 737400], bottom tick price=10487411812500898840576.000000 WETH for WBTC
        0.00 WBTC locked, potentially worth 2192571416.26 WETH
ticks=[737400, 737460], bottom tick price=10550522269956185260032.000000 WETH for WBTC
        0.00 WBTC locked, potentially worth 2199158677.10 WETH
ticks=[737460, 737520], bottom tick price=10614012509374023467008.000000 WETH for WBTC
        0.00 WBTC locked, potentially worth 2205765728.41 WETH
ticks=[737520, 737580], bottom tick price=10677884816181340143616.000000 WETH for WBTC
        0.00 WBTC locked, potentially worth 2212392629.63 WETH
ticks=[737580, 737640], bottom tick price=1074214

,tick,liquidity,price,pool_id,amount0,amount1,total-amount0,total-amount1
0,-887220,71124428655530,2.954278e-49,WBTC/WETH-0.3,3.919597e+22,1.161437e-26,0.000000,1.161437e-26
1,-887160,80064092962998,2.972056e-49,WBTC/WETH-0.3,8.306892e+22,2.476273e-26,0.000000,3.637710e-26
2,-887100,0,2.989942e-49,WBTC/WETH-0.3,8.282010e+22,2.483712e-26,0.000000,6.121422e-26
3,-887040,0,3.007934e-49,WBTC/WETH-0.3,8.257203e+22,2.491174e-26,0.000000,8.612596e-26
4,-886980,0,3.026035e-49,WBTC/WETH-0.3,8.232470e+22,2.498659e-26,0.000000,1.111125e-25
...,...,...,...,...,...,...,...,...
29570,886980,0,3.304654e+28,WBTC/WETH-0.3,1.174232e-16,3.892088e+12,4526.235173,7.694693e+04
29571,887040,0,3.324541e+28,WBTC/WETH-0.3,1.170714e-16,3.903781e+12,4526.235173,7.694693e+04
29572,887100,0,3.344547e+28,WBTC/WETH-0.3,1.167208e-16,3.915509e+12,4526.235173,7.694693e+04
29573,887160,0,3.364674e+28,WBTC/WETH-0.3,1.163711e-16,3.927273e+12,4526.235173,7.694693e+04


In [13]:
wbtc_WETH_3000_df.to_csv('Ticks_Anylsis_WBTC_WETH.csv',index=False)

In [77]:
dai_usdc100_df = getLiguidityDistribuation(dai_usdc100_id,"DAI/USDC-0.01")
dai_usdc100_df

Streaming output truncated to the last 5000 lines.
ticks=[884772, 884773], bottom tick price=264995564233997151366684350700868877448298912284672.000000 USDC for DAI
        0.00 DAI locked, potentially worth 0.00 USDC
ticks=[884773, 884774], bottom tick price=265022063790420548676749114262606806800717842481152.000000 USDC for DAI
        0.00 DAI locked, potentially worth 166042017372910680801280.00 USDC
ticks=[884774, 884775], bottom tick price=265048565996799566061975954903360038807303754874880.000000 USDC for DAI
        0.00 DAI locked, potentially worth 0.00 USDC
ticks=[884775, 884776], bottom tick price=265075070853399259892399358503909798244670676926464.000000 USDC for DAI
        0.00 DAI locked, potentially worth 166058621574556533391360.00 USDC
ticks=[884776, 884777], bottom tick price=265101578360484561922929206109174225157520734814208.000000 USDC for DAI
        0.00 DAI locked, potentially worth 0.00 USDC
ticks=[884777, 884778], bottom tick price=26512808851832061160034972

,tick,liquidity,price,pool_id,amount0,amount1,total-amount0,total-amount1
0,-887272,101999654137826,2.938957e-27,DAI/USDC-0.01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,-887271,0,2.939251e-27,DAI/USDC-0.01,1.881303e+11,5.529620e-16,0.000000e+00,5.529620e-16
2,-887270,0,2.939545e-27,DAI/USDC-0.01,0.000000e+00,0.000000e+00,0.000000e+00,5.529620e-16
3,-887269,0,2.939839e-27,DAI/USDC-0.01,1.881115e+11,5.530173e-16,0.000000e+00,1.105979e-15
4,-887268,0,2.940133e-27,DAI/USDC-0.01,0.000000e+00,0.000000e+00,0.000000e+00,1.105979e-15
...,...,...,...,...,...,...,...,...
1774540,887268,0,3.401207e+50,DAI/USDC-0.01,0.000000e+00,0.000000e+00,4.106011e+08,4.115893e+08
1774541,887269,0,3.401547e+50,DAI/USDC-0.01,5.530173e-28,1.881115e+23,4.106011e+08,4.115893e+08
1774542,887270,0,3.401887e+50,DAI/USDC-0.01,0.000000e+00,0.000000e+00,4.106011e+08,4.115893e+08
1774543,887271,0,3.402228e+50,DAI/USDC-0.01,5.529620e-28,1.881303e+23,4.106011e+08,4.115893e+08


In [ ]:
dai_usdc100_df.to_csv('Ticks_Anylsis_DAI_USDC.csv',index=False)

In [78]:
#result = pd.concat([usdc_WETH_3000_df,dai_usdc100_df,wbtc_WETH_3000_df])
result = pd.concat([usdc_WETH_3000_df,wbtc_WETH_3000_df])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1833695 entries, 0 to 29574
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   tick           int64  
 1   liquidity      object 
 2   price          float64
 3   pool_id        object 
 4   amount0        float64
 5   amount1        float64
 6   total-amount0  float64
 7   total-amount1  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 125.9+ MB


In [ ]:
result.to_csv('Ticks_Anylsis.csv',index=False)